In [2]:
# 基础代码引入
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI

# 加载当前目录下的.env文件
# load_dotenv()
# load_dotenv(override=True) 会重新读取.env
load_dotenv(override=True)

# 现在可以像访问普通环境变量一样访问.env文件中的变量了
api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')

print(api_key)
print(base_url)

chat = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [ ]:
# chains 就是一系列的函数调用

In [5]:
from langchain.chains import LLMChain
from langchain_core.prompts import PromptTemplate

# 原始方式：不用lcel，利用prompt
prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)
llm = LLMChain(llm=chat, prompt=prompt)
llm.invoke("boss")

{'adjective': 'boss',
 'text': "Sure, I'd be happy to share a light-hearted boss joke with you:\n\nWhy did the boss bring a ladder to work?\n\nBecause he wanted to step up his game!\n\nRemember, it's always important to maintain a healthy sense of humor in the workplace. Just be sure to keep jokes appropriate and considerate of your coworkers' feelings."}

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 用lcel，利用prompt
prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
output_parser = StrOutputParser()

chain = prompt | chat | output_parser

chain.invoke({"topic": "ice cream"})

'Sure, here\'s a short joke about ice cream:\n\nWhy did the ice cream go to school?\n\nBecause it wanted to be a "scoop" student!'

In [10]:
# from langchain_openai import OpenAI
# 
# client = OpenAI(
#     api_key=api_key,
#     base_url="https://api.moonshot.cn/v1",
# )

In [3]:
from openai import OpenAI

client = OpenAI(
    api_key=api_key,
    base_url="https://api.moonshot.cn/v1",
)

# completion = client.chat.completions.create(
#     model = "moonshot-v1-8k",
#     messages = [
#         {"role": "system", "content": "你是 Kimi，由 Moonshot AI 提供的人工智能助手，你更擅长中文和英文的对话。你会为用户提供安全，有帮助，准确的回答。同时，你会拒绝一切涉及恐怖主义，种族歧视，黄色暴力等问题的回答。Moonshot AI 为专有名词，不可翻译成其他语言。"},
#         {"role": "user", "content": "你好，我叫李雷，1+1等于多少？"}
#     ],
#     temperature = 0.3,
# )
# 
# print(completion.choices[0].message.content)

In [ ]:
# from langchain_community.llms.moonshot import Moonshot
# 
# llm = Moonshot()

In [11]:
from typing import Iterator


# 流式 输出
def stream_chat_model(messages: list[dict]) -> Iterator[str]:
    stream = client.chat.completions.create(
        model="moonshot-v1-8k",
        messages=messages,
        stream=True,
    )
    for response in stream:
        content = response.choices[0].delta.content
        if content is not None:
            yield content


def stream_chain(topic: str) -> Iterator[str]:
    prompt_value = prompt.format(adjective=topic)
    return stream_chat_model([{"role": "user", "content": prompt_value}])


for chunk in stream_chain("中国人民最幸福"):
    print(chunk, end="", flush=True)

As an AI language model, I am committed to promoting a positive and respectful environment. I cannot provide a joke that may be offensive or disrespectful to any group of people, including the Chinese people. However, I'd be happy to share a light-hearted, culturally neutral joke with you. Here's one:

Why don't scientists trust atoms?
Because they make up everything!

In [9]:
for chunk in chain.stream({"topic": "ice cream"}):
    print(chunk, end="", flush=True)

NameError: name 'chain' is not defined

In [8]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

# 组合多个chain
prompt1 = ChatPromptTemplate.from_template("what is the city {person} is from?")
prompt2 = ChatPromptTemplate.from_template(
    "what country is the city {city} in? respond in {language}"
)

chain1 = prompt1 | chat | StrOutputParser()

chain2 = (
        {"city": chain1, "language": itemgetter("language")}
        | prompt2
        | chat
        | StrOutputParser()
)

chain2.invoke({"person": "obama", "language": "chinese"})

'为了找出巴拉克·奥巴马所在的城市，我将遵循以下步骤：\n\n1. 回想一下，巴拉克·奥巴马是美国的前总统。\n2. 记住他出生在夏威夷的檀香山。\n3. 确定檀香山是奥巴马出生并度过早年时光的城市。\n\n所以，巴拉克·奥巴马来自夏威夷的檀香山。\n\n在中文中的回应是：\n\n为了找出巴拉克·奥巴马所在的城市，我将遵循以下步骤：\n\n1. 回想一下，巴拉克·奥巴马是美国的前总统。\n2. 记住他出生在夏威夷的檀香山。\n3. 确定檀香山是奥巴马出生并度过早年时光的城市。\n\n所以，巴拉克·奥巴马来自夏威夷的檀香山。'